In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Flatten, BatchNormalization
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, median_absolute_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

file_path = "train_FD004_processed.csv"
df = pd.read_csv(file_path)

if df.isna().any().any():
    print("NaN values found in dataset. Removing rows with NaN.")
    df = df.dropna()

max_rul = 130 
df['RUL'] = df['RUL'].clip(upper=max_rul)

WINDOW_SIZE = 30
FEATURE_COLS = [col for col in df.columns if col.startswith('op_setting_') or col.startswith('sensor_measurement_')]
TARGET_COL = 'RUL'

variances = df[FEATURE_COLS].var()
selected_features = variances[variances > 0.01].index.tolist()  # Keep features with variance > 0.01
print(f"Selected features: {selected_features}")
df = df[['unit_number'] + selected_features + [TARGET_COL]]

scaler_features = StandardScaler()
scaler_target = StandardScaler()

df[selected_features] = scaler_features.fit_transform(df[selected_features])

# Normalize target (RUL)
df[TARGET_COL] = scaler_target.fit_transform(df[[TARGET_COL]])

if df.isna().any().any():
    print("NaN values found after normalization. Removing rows with NaN.")
    df = df.dropna()

def create_sequences(data, window_size, feature_cols, target_col):
    X, y = [], []
    for unit in data['unit_number'].unique():
        unit_data = data[data['unit_number'] == unit]
        feature_data = unit_data[feature_cols].values
        target_data = unit_data[target_col].values
        for i in range(len(unit_data) - window_size):
            X.append(feature_data[i:i+window_size])
            y.append(target_data[i+window_size])
    return np.array(X), np.array(y)

X, y = create_sequences(df, WINDOW_SIZE, selected_features, TARGET_COL)

if np.any(np.isnan(X)) or np.any(np.isnan(y)):
    print("NaN values found in sequences. Removing affected samples.")
    mask = ~np.isnan(X).any(axis=(1, 2)) & ~np.isnan(y)
    X, y = X[mask], y[mask]

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

input_shape = X.shape[1:]  # (window_size, num_features)
inputs = Input(shape=input_shape)

def transformer_block(x, num_heads, key_dim, ff_dim, dropout_rate=0.1):

    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x, x)
    attn_output = Dropout(dropout_rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(x + attn_output)

    ffn_output = Dense(ff_dim, activation='relu')(out1)
    ffn_output = Dense(input_shape[-1])(ffn_output)
    ffn_output = Dropout(dropout_rate)(ffn_output)
    return LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

x = inputs
x = transformer_block(x, num_heads=4, key_dim=32, ff_dim=128, dropout_rate=0.2)
x = transformer_block(x, num_heads=4, key_dim=32, ff_dim=128, dropout_rate=0.2)  # Added second block
x = transformer_block(x, num_heads=4, key_dim=32, ff_dim=128, dropout_rate=0.2)  # Added third block

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
outputs = Dense(1)(x)

model = Model(inputs, outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0)  # Reduced learning rate, gradient clipping
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=16,  # Reduced batch size for stability
    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)

# Predict on validation set
y_pred = model.predict(X_val)

# Check for NaN in predictions
if np.any(np.isnan(y_pred)):
    print("NaN values found in predictions. Replacing with 0.")
    y_pred = np.nan_to_num(y_pred, 0)

# Inverse transform the predictions and true values
y_val_inv = scaler_target.inverse_transform(y_val.reshape(-1, 1)).flatten()
y_pred_inv = scaler_target.inverse_transform(y_pred).flatten()

# Check for NaN after inverse transformation
if np.any(np.isnan(y_val_inv)) or np.any(np.isnan(y_pred_inv)):
    print("NaN values found after inverse transformation. Removing affected samples.")
    mask = ~np.isnan(y_val_inv) & ~np.isnan(y_pred_inv)
    y_val_inv, y_pred_inv = y_val_inv[mask], y_pred_inv[mask]

# Calculate all test parameters
mse = mean_squared_error(y_val_inv, y_pred_inv)
mae = mean_absolute_error(y_val_inv, y_pred_inv)
rmse = np.sqrt(mse)
r2 = r2_score(y_val_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_val_inv, y_pred_inv)
medae = median_absolute_error(y_val_inv, y_pred_inv)

# Print all test parameters
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"Median Absolute Error: {medae:.4f}")

# Visualizations
# 1. Training History Plot
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Transformer Model Loss During Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('transformer_training_loss_plot.png')
plt.close()

# 2. Predicted vs Actual RUL Scatter Plot
plt.figure(figsize=(10, 6))
plt.scatter(y_val_inv, y_pred_inv, alpha=0.5, color='blue')
plt.plot([y_val_inv.min(), y_val_inv.max()], [y_val_inv.min(), y_val_inv.max()], 'r--', lw=2, label='Ideal Fit')
plt.title('Predicted vs Actual RUL (Transformer)')
plt.xlabel('Actual RUL')
plt.ylabel('Predicted RUL')
plt.legend()
plt.grid(True)
plt.savefig('transformer_predicted_vs_actual_rul.png')
plt.close()

# 3. Feature Importance Bar Plot
feature_importance = pd.Series(variances.loc[selected_features], index=selected_features)
plt.figure(figsize=(12, 6))
feature_importance.sort_values(ascending=False).plot(kind='bar')
plt.title('Feature Importance Based on Variance')
plt.xlabel('Features')
plt.ylabel('Variance')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('transformer_feature_importance_plot.png')
plt.close()

# Save the model
model.save("enhanced_transformer_rul_model.keras")

NaN values found in dataset. Removing rows with NaN.
Selected features: ['op_setting_1', 'op_setting_2', 'op_setting_3', 'sensor_measurement_1', 'sensor_measurement_2', 'sensor_measurement_3', 'sensor_measurement_4', 'sensor_measurement_5', 'sensor_measurement_6', 'sensor_measurement_7', 'sensor_measurement_8', 'sensor_measurement_9', 'sensor_measurement_10', 'sensor_measurement_11', 'sensor_measurement_12', 'sensor_measurement_13', 'sensor_measurement_14', 'sensor_measurement_15', 'sensor_measurement_16', 'sensor_measurement_17', 'sensor_measurement_18', 'sensor_measurement_19', 'sensor_measurement_20', 'sensor_measurement_21']


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 30, 24)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 30, 24)    │     12,696 │ input_layer[0][0… │
│ (MultiHeadAttentio… │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 30, 24)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 30, 24)    │          0 │ input_layer[0][0… │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 30, 24)    │         48 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 30, 128)   │      3,200 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 30, 24)    │      3,096 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 30, 24)    │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 30, 24)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 30, 24)    │         48 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 30, 24)    │     12,696 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 30, 24)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 30, 24)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 30, 24)    │         48 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 30, 128)   │      3,200 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 30, 24)    │      3,096 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 30, 24)    │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 30, 24)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 30, 24)    │         48 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 158,641 (619.69 KB)

 Trainable params: 158,257 (618.19 KB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 40s 26ms/step - loss: 1.8920 - mae: 1.0714 - val_loss: 0.5432 - val_mae: 0.5603 - learning_rate: 5.0000e-04
Epoch 2/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.5226 - mae: 0.5689 - val_loss: 0.3626 - val_mae: 0.4273 - learning_rate: 5.0000e-04
Epoch 3/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.3693 - mae: 0.4739 - val_loss: 0.3053 - val_mae: 0.3617 - learning_rate: 5.0000e-04
Epoch 4/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.3301 - mae: 0.4461 - val_loss: 0.2615 - val_mae: 0.3651 - learning_rate: 5.0000e-04
Epoch 5/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.3024 - mae: 0.4245 - val_loss: 0.2554 - val_mae: 0.3528 - learning_rate: 5.0000e-04
Epoch 6/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.2983 - mae: 0.4200 - val_loss: 0.2862 - val_mae: 0.3512 - learning_rate: 5.0000e-04
Epoch 7/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.2979 - mae: 0.4199 - val_loss: 0.2276 - val_mae: 0.3613 - le